In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
import sqlite3

In [25]:
#Alphavantage URL is defined below to use the Alphavantage API for data collection
api_key = "M0H1JZNA6VMOG30H"
alphavantage_url = "https://www.alphavantage.co/query?"

# Bank of Canada API URL is set-up for fetching inflation data
inlflation_url = "https://www.bankofcanada.ca/valet/observations/V41690914/json"
response_inflation = requests.get(inlflation_url)

In [32]:
# Since data was insufficiently available on Yahoo Finance, iShares ETF data for the index has been pulled from Alphavantage instead
url_tsx = (f"{alphavantage_url}function=TIME_SERIES_DAILY&symbol=XIU.TO&outputsize=full&extended_hours=false&apikey={api_key}&datatype=csv")
url_ttfs = (f"{alphavantage_url}function=TIME_SERIES_DAILY&symbol=XFN.TO&outputsize=full&extended_hours=false&apikey={api_key}&datatype=csv")
url_tten = (f"{alphavantage_url}function=TIME_SERIES_DAILY&symbol=XEG.TO&outputsize=full&extended_hours=false&apikey={api_key}&datatype=csv")
url_ttin = (f"{alphavantage_url}function=TIME_SERIES_DAILY&symbol=ZIN.TO&outputsize=full&extended_hours=false&apikey={api_key}&datatype=csv")
url_ttma = (f"{alphavantage_url}function=TIME_SERIES_DAILY&symbol=XMA.TO&outputsize=full&extended_hours=false&apikey={api_key}&datatype=csv")
url_ttit = (f"{alphavantage_url}function=TIME_SERIES_DAILY&symbol=XIT.TO&outputsize=full&extended_hours=false&apikey={api_key}&datatype=csv")

In [33]:
# The read_csv function is used to read the data via the APIs above, the data is converted to dataframes and varaibles relative to the sectors are defined below.
df_tsx = pd.read_csv(url_tsx)
df_financial = pd.read_csv(url_ttfs)
df_energy = pd.read_csv(url_tten)
df_industrials = pd.read_csv(url_ttin)
df_materials = pd.read_csv(url_ttma)
df_technology = pd.read_csv(url_ttit)

In [29]:
# Data is saved to CSVs for analysis
df_tsx.to_csv("TSX.csv")
df_financial.to_csv("Financial Sector.csv")
df_energy.to_csv("Energy Sector.csv")
df_industrials.to_csv("Industrials Sector.csv")
df_materials.to_csv("Materials Sector.csv")
df_technology.to_csv("IT Sector.csv")

In [34]:
# Timestamp column in csv files is converted to datetime format, to help clean the data
df_tsx['timestamp'] = pd.to_datetime(df_tsx['timestamp'])
df_financial['timestamp'] = pd.to_datetime(df_financial['timestamp'])
df_energy['timestamp'] = pd.to_datetime(df_energy['timestamp'])
df_industrials['timestamp'] = pd.to_datetime(df_industrials['timestamp'])
df_materials['timestamp'] = pd.to_datetime(df_materials['timestamp'])
df_technology['timestamp'] = pd.to_datetime(df_technology['timestamp'])

# Data is cleaned as per the time period of the analysis by defining the start and end dates below
start_date = '2020-01-01'
end_date = '2024-12-31'

# The dataFrame is filtered for the above specified date range
clean_df_tsx = df_tsx[(df_tsx['timestamp'] >= start_date) & (df_tsx['timestamp'] <= end_date)]
clean_df_financial = df_financial[(df_financial['timestamp'] >= start_date) & (df_financial['timestamp'] <= end_date)]
clean_df_energy = df_energy[(df_energy['timestamp'] >= start_date) & (df_energy['timestamp'] <= end_date)]
clean_df_industrials = df_industrials[(df_industrials['timestamp'] >= start_date) & (df_industrials['timestamp'] <= end_date)]
clean_df_materials = df_materials[(df_materials['timestamp'] >= start_date) & (df_materials['timestamp'] <= end_date)]
clean_df_technology = df_technology[(df_technology['timestamp'] >= start_date) & (df_technology['timestamp'] <= end_date)]


In [12]:
# The cleaned data is now saved to CSVs
clean_df_tsx.to_csv("TSX.csv")
clean_df_financial.to_csv("Financial Sector.csv")
clean_df_energy.to_csv("Energy Sector.csv")
clean_df_industrials.to_csv("Industrials Sector.csv")
clean_df_materials.to_csv("Materials Sector.csv")
clean_df_technology.to_csv("IT Sector.csv")

In [35]:
# Inflation data is sourced via the Bank of Canada API set up earlier
data = response_inflation.json()

# Extract observations (GDP data)
observations = data['observations']

cpi_data = pd.DataFrame(observations)

# Extract relevant fields: date and CPI value
cpi_data['date'] = pd.to_datetime(cpi_data['d'])  # Convert 'd' to datetime - d column stores the relevant dates for the required data
cpi_data['CPI'] = cpi_data['V41690914'].apply(lambda x: x['v']).astype(float)  # Extract 'v' from 'V41690914', as this column stores the cpi data

# Keep only relevant columns
cpi_data = cpi_data[['date', 'CPI']]

# Filter data for 2019 onwards
cpi_data = cpi_data[cpi_data['date'] >= '1999-01-01']

# The pct_change(12) function is used on the monthly CPI data, to compound it over 12 months and obtain the annualized infaltion 
cpi_data['Inflation Rate (%)'] = cpi_data['CPI'].pct_change(12) * 100

cpi_data.to_csv("canadian_inflation.csv")

In [3]:
conn = sqlite3.connect("tsx_analysis.db")
cursor = conn.cursor()

In [3]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS tsx_daily (
    date TEXT PRIMARY KEY,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER
)
""")

In [10]:
# Reading the TSX data saved earlier for further analysis

tsx_data = pd.read_csv("Resources\Alphavantage\TSX.csv")
energy_sector = pd.read_csv("Resources\Alphavantage\Energy Sector.csv")
financial_sector = pd.read_csv("Resources\Alphavantage\Financial Sector.csv")
industrial_sector = pd.read_csv("Resources\Alphavantage\Industrials Sector.csv")
IT_sector = pd.read_csv("Resources\Alphavantage\IT Sector.csv")
materials_sector = pd.read_csv("Resources\Alphavantage\Materials Sector.csv")


In [4]:
# Reading the macroeconomic data collected earlier
fx_data = pd.read_csv("Resources\CADUSD_Historical_Monthly.csv")
inflation_data = pd.read_csv("Resources\Bank of Canada\canadian_inflation_annualized.csv")


In [37]:
# Loading the data to SQLite table
tsx_data.to_sql("tsx_daily", conn, if_exists="replace", index=False)
energy_sector.to_sql("energy_sector", conn, if_exists="replace", index=False)
financial_sector.to_sql("financial_sector", conn, if_exists="replace", index=False)
industrial_sector.to_sql("industrial_sector", conn, if_exists="replace", index=False)
IT_sector.to_sql("IT_sector", conn, if_exists="replace", index=False)
materials_sector.to_sql("materials_sector", conn, if_exists="replace", index=False)


1250

In [6]:
fx_data.to_sql("fx_data", conn, if_exists="replace", index=False)
inflation_data.to_sql("CPI_Inflation", conn, if_exists="replace", index=False)


71

In [18]:
inflation_data.rename(columns={'date': 'timestamp'}, inplace=True)

In [13]:
# The timestamp column is defined as datetime
tsx_data['timestamp'] = pd.to_datetime(tsx_data['timestamp'])
energy_sector['timestamp'] = pd.to_datetime(energy_sector['timestamp'])
financial_sector['timestamp'] = pd.to_datetime(financial_sector['timestamp'])
industrial_sector['timestamp'] = pd.to_datetime(industrial_sector['timestamp'])
IT_sector['timestamp'] = pd.to_datetime(IT_sector['timestamp'])
materials_sector['timestamp'] = pd.to_datetime(materials_sector['timestamp'])

NameError: name 'tsx_data' is not defined

In [19]:
# Similarly to TSX data earlier, macroeconomic data timestamp is also coverted to Datetime format

inflation_data['timestamp'] = pd.to_datetime(inflation_data['timestamp'])
fx_data['timestamp'] = pd.to_datetime(fx_data['timestamp'])

In [46]:
# Data is cleaned to remove blank columns

for df in [tsx_data, energy_sector, financial_sector, industrial_sector, IT_sector, materials_sector]:
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)

In [48]:
# The cleaned data is updated to SQLite

tsx_data.to_sql("tsx_daily", conn, if_exists="replace", index=False)
energy_sector.to_sql("energy_sector", conn, if_exists="replace", index=False)
financial_sector.to_sql("financial_sector", conn, if_exists="replace", index=False)
industrial_sector.to_sql("industrial_sector", conn, if_exists="replace", index=False)
IT_sector.to_sql("IT_sector", conn, if_exists="replace", index=False)
materials_sector.to_sql("materials_sector", conn, if_exists="replace", index=False)

1250

In [16]:
tsx_query = pd.read_sql("SELECT * FROM tsx_daily LIMIT 5;", conn)
print(tsx_query)

             timestamp   open   high    low  close    volume
0  2024-12-20 00:00:00  36.75  37.45  36.73  37.26  31832200
1  2024-12-19 00:00:00  37.34  37.36  36.98  37.00   4927900
2  2024-12-18 00:00:00  38.05  38.07  37.16  37.23   5660300
3  2024-12-17 00:00:00  37.99  38.11  37.92  38.09   3031700
4  2024-12-16 00:00:00  38.27  38.27  38.08  38.10   1723400


In [20]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


[('tsx_daily',), ('energy_sector',), ('financial_sector',), ('industrial_sector',), ('IT_sector',), ('materials_sector',), ('fx_data',), ('CPI_Inflation',)]
